In [16]:
import json
from tqdm import tqdm
from malaysian_sft import accept as accept_fn

In [17]:
rejected_personality = [
    'menteri',
    'minister'
]

accept = []
with open('generated-malaysian-politician-hansard-style.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        personality = l['personality'].strip().lower()
        if len(personality) < 2:
            continue
        
        if any([r in personality for r in rejected_personality]):
            continue
            
        accept.append(l)
        
len(accept)

300459it [00:01, 167913.71it/s]


278016

In [46]:
filtered = []
accept_politician = [
    'tuan',
    'dato',
    'datuk',
    'dr',
    'a/l',
    'a/p',
    'muhammad',
    'bin',
    'binti',
    'mohd',
    'haji',
    'datin',
]
for a in tqdm(accept):
    if len(a['answer'].split()) < 20:
        continue
    
    if len(a['question'].split()) < 5:
        continue
        
    politician = a['politician'].lower()
    
    if all([r not in politician for r in accept_politician]):
        continue
        
    if not accept_fn(a['answer']):
        continue
    
    filtered.append(a)
    
len(filtered)

100%|█████████████████████████████████| 278016/278016 [00:44<00:00, 6277.51it/s]


220778

In [47]:
filtered[0]

{'url': 'https://www.parlimen.gov.my/files/hindex/pdf/DR-01072024.pdf',
 'date': '2024-07-01',
 'actual_no_pages': [9, 10],
 'no_pages': [3, 4],
 'question': 'Bagaimana Kerajaan Persekutuan dapat mempertahankan hak-hak pekebun dan melindungi pekebun-pekebun tersebut tidak ditindas oleh syarikat besar tersebut?',
 'answer': 'Hal tanah adalah di bawah kerajaan negeri. Jadi, sebarang masalah hal tanah ini kita harapkan berlaku perundingan antara mereka yang meneroka ataupun menanam itu dengan kerajaan negeri. Inilah yang kita mengharapkan sebab kita tahu yang terlibat menanam secara sah, 8,782 dan bilangan yang meneroka itu adalah 186 dan sekitar dua persen.',
 'politician': 'Datuk Seri Haji Mohamad bin Sabu',
 'personality': 'Helpful',
 'topic': 'Agriculture'}

In [48]:
templates = [
    'anda adalah {name} yg mempunyai personality {personality} yg sedang discuss ttg {topic} didalam sesi parlimen',
    'you are {name} with personality {personality} discussing about {topic} in parliament session',
    'Anda adalah seorang pembantu AI yang mempunyai watak {name} dengan personaliti {personality} di dalam sesi parlimen'
]

In [49]:
import random

data = []
for f in filtered:
    q = f['question']
    a = f['answer']
    system = random.choice(templates).format(name = f['politician'], personality = f['personality'], topic = f['topic'])
    if random.random() > 0.7:
        q = f'{system}\n\n{q}'
        system = None
    
    data.append({
        'prompt_input': system,
        'input': q,
        'output': a,
    })

In [50]:
len(data)

220778

In [51]:
data[-2]

{'prompt_input': 'anda adalah Dr. Kelvin Yii Lee Wuen yg mempunyai personality Member of Parliament yg sedang discuss ttg Mental Health didalam sesi parlimen',
 'input': 'What is the role of caregivers in supporting individuals with mental health issues?',
 'output': 'Jadi, apakah peranan kementerian untuk kita for this group of people, the care givers? Jika kita menjaga care givers dengan baik, mereka akan menjaga pesakit kita dengan lebih baik.'}

In [52]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

,prompt_input,input,output
0,you are Datuk Seri Haji Mohamad bin Sabu with ...,Bagaimana Kerajaan Persekutuan dapat mempertah...,Hal tanah adalah di bawah kerajaan negeri. Jad...
1,Anda adalah seorang pembantu AI yang mempunyai...,Sejauh mana kerajaan mengambil kesempatan deng...,Kenapa China? Sebab China makan durian. Negara...
2,Anda adalah seorang pembantu AI yang mempunyai...,Apakah mekanisme kerajaan bagi mengawal penggu...,"Soal pest control dan sebagainya itu, itu dala..."
3,Anda adalah seorang pembantu AI yang mempunyai...,Negeri-negeri seterusnya yang disasarkan oleh ...,Kerajaan terus komited untuk menoktahkan kemis...
4,you are Dato Hajjah Hanifah Hajar Taib with pe...,Kenapa data kemiskinan tegar berbeza antara IC...,"Saya akan rujuk kepada pihak DOSM, apakah dari..."


In [53]:
df.to_parquet('prepared-malaysian-politician-hansard-style.parquet')

In [54]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-malaysian-politician-hansard-style.parquet",
    path_in_repo='data/malaysian_politician_hansard_style-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-malaysian-politician-hansard-style.parquet:   0%|          | 0.00/60.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/fa3b9432705bc90144ab0480cfe60dc76518f77e', commit_message='Upload data/malaysian_politician_hansard_style-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='fa3b9432705bc90144ab0480cfe60dc76518f77e', pr_url=None, pr_revision=None, pr_num=None)